## Infer point

Try to infer single endmember.

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

from model.inference import *
from model.hapke_model import get_USGS_r_mixed_hapke_estimate
from preprocessing.generate_USGS_data import generate_image
from utils.plotting import *
from utils.access_data import *
from utils.constants import *

def get_rmse(a, b): 
    return np.sqrt(np.mean((a - b)**2))

def print_error(m_actual, D_actual, m_est, D_est): 
    m_rmse = str(round(get_rmse(m_actual, m_est), 2))
    D_rmse = str(round(get_rmse(D_actual, D_est), 2))
    print(str(np.array(D_actual)) + ", "  + str(m_est) + ", " + str(m_rmse) + ", " + str(D_est) + ", " + str(D_rmse))
    return m_rmse, D_rmse

plot_endmembers(CRISM_match=False)

In [ ]:
def test_inference(m_sample, D_sample, name):
    true_m = convert_arr_to_dict(m_sample)
    true_D = convert_arr_to_dict(D_sample)
    r_actual = get_USGS_r_mixed_hapke_estimate(m=true_m, D=true_D)
    est_m, est_D = infer_datapoint(d_seeds_index=[r_actual, 2, 0],
                                   iterations=NUM_ITERATIONS, 
                                   C=10, 
                                   V=GRAIN_SIZE_COVARIANCE)
    m_rmse, D_rmse=print_error(m_sample, D_sample, est_m, est_D)
    wavelengths = get_USGS_wavelengths()
    r_est = get_USGS_r_mixed_hapke_estimate(convert_arr_to_dict(est_m),  convert_arr_to_dict(est_D))
    fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(FIG_WIDTH, FIG_HEIGHT), dpi=DPI)
    ax.plot(wavelengths, r_est, label = "Estimated", color = "orange")
    ax.plot(wavelengths, r_actual, label = "Actual", color="blue")
    ax.set_xlabel("Wavelength")
    ax.set_ylabel("Reflectance")
    ax.legend()
    ax.set_title("RMSE M: " + str(m_rmse) + ", D: " + str(D_rmse))
    plt.ylim((0, 1)) 
    plt.savefig("../output/tests/" + str(name) + ".pdf") 

In [ ]:

# ["augite", "enstatite", "labradorite",  "olivine (Fo51)"]
m_sample = [0.6, 0.05,.05,0.4]
D_sample = [200, 300, 100, 200]
NAME = "mixed_7_1000_iters"
GRAIN_SIZE_COVARIANCE = 50
NUM_ITERATIONS = 1000
test_inference(m_sample, D_sample, NAME)

## Synthetic Image Generation
Create synthetic image resembling some realistic geology to have single, useful test image.

In [ ]:
#Create 2D Numpy Matrix based on circles
from matplotlib.lines import Line2D
import matplotlib
def points_in_circle_np(radius, x0=0, y0=0):
    """
    Get X, Y coords for given circle
    """
    x_ = np.arange(x0 - radius - 1, x0 + radius + 1, dtype=int)
    y_ = np.arange(y0 - radius - 1, y0 + radius + 1, dtype=int)
    x, y = np.where((x_[:,np.newaxis] - x0)**2 + (y_ - y0)**2 <= radius**2)
    # x, y = np.where((np.hypot((x_-x0)[:,np.newaxis], y_-y0)<= radius)) # alternative implementation
    for x, y in zip(x_[x], y_[y]):
        yield x, y

        

img_width = 24
img_height = 24
color_img = np.zeros((img_width,img_height,3))
test_img = np.zeros((img_width,img_height,1))

center_X = int(img_height/2)
center_Y = int(img_width/2)
outer_circle = points_in_circle_np(9, x0=center_X, y0=center_Y)
medium_circle =  points_in_circle_np(6, x0=center_X, y0=center_Y)
inner_circle =  points_in_circle_np(3, x0=center_X+1, y0=center_Y+1)
lower_ring = points_in_circle_np(11, x0=center_X, y0=center_Y-1)

circles = [lower_ring, 
           outer_circle, 
           medium_circle,
           inner_circle]
LR_C = [255, 102, 255]
O_C = [102, 255, 204]
M_C = [51, 51, 255]
I_C = [51, 102, 255]

RGB_MAP = {0: LR_C,
          1: O_C,
          2: M_C,
          3: I_C}
DEFAULT_COLOR =  [153, 51, 51]

for row_index, row in enumerate(color_img):
    for col_index, col in enumerate(row):
        color_img[row_index,col_index] = DEFAULT_COLOR
        test_img[row_index,col_index] = 0
            
for i, circle in enumerate(circles):
    for point in circle:
        x=point[0]
        y=point[1] 
        if i != 0:
            color_img[x,y] = RGB_MAP[i]
            test_img[x,y] = i + 1
        else:
            # only do lower, ring so threshold x 
            if x >= int(img_height*0.6):
                color_img[x,y] = RGB_MAP[i]
                test_img[x,y] = i + 1
# Plot image
figure, ax = plt.subplots(1)
color_img=np.array(color_img,np.int32)
            
plt.imshow(color_img)

a = Line2D([0], [0], color='#%02x%02x%02x' % tuple(DEFAULT_COLOR), lw=4) 
clines = [a] 
for c in list(RGB_MAP.values()):  
    hex_color='#%02x%02x%02x' % tuple(c)
    a = Line2D([0], [0], color=hex_color, lw=4) 
    clines.append(a)

ax.legend(clines, [ "Background", "Lower ring", "Outer", "Inner", "Peak"],loc=(1.1,0.5))

plt.savefig(PREPROCESSED_DATA + "SYNTHETIC/visual.pdf", bbox_inches='tight')

In [ ]:
# ["augite", "enstatite", "labradorite",  "olivine (Fo51)"]
testing_regions = ["b", "l", "o", "i", "p"]
m_type_map = { 0:"b", 1:"l", 2 :"o", 3 :"i", 4:"p"}
region_ms = {"b": [0.3, 0.2,.1,0.4],
                "l": [0.7, 0, 0.3,0],
                "o": [0, 0.6,0.4,0],
                "i": [0.8, 0.2, 0, 0],
                "p": [0.3, 0, 0, 0.7]}
region_Ds = {"b": [200, 300, 100, 200],
                "l": [200, 300, 100, 200],
                "o": [200, 300, 100, 200],
                "i": [200, 300, 100, 200],
                "p": [200, 300, 100, 200]}
region_Rs = {}

# Mix each endmember; and then just add a bit of noise when creating image
for ttype in testing_regions: 
    m_map = {}
    D_map = {}
    for index, endmember in enumerate(USGS_PURE_ENDMEMBERS):
        m_map[endmember] = region_ms[ttype][index]
        D_map[endmember] = region_Ds[ttype][index]
    r = get_USGS_r_mixed_hapke_estimate(m_map, D_map)
    region_Rs[ttype] = r
    
with open(R_DIR + "../wavelengths.pickle", 'rb') as handle:
    wavelengths = pickle.load(handle)

    
r_image = np.zeros((img_width,img_height,len(wavelengths)))
m_image = np.zeros((img_width,img_height,len(USGS_PURE_ENDMEMBERS)))
D_image = np.zeros((img_width,img_height,len(USGS_PURE_ENDMEMBERS)))

# Fill test image with synthetically mixed spectra
for row_index, row in enumerate(test_img):
    for col_index, col in enumerate(row):
        ttype_index = test_img[row_index, col_index]
        ttype = testing_regions[int(ttype_index)]
        cur_R = region_Rs[ttype]
        # Add noise here
#         noise = np.random.normal(loc=0,
#                                      scale=noise_scale,
#                                      size=len(wavelengths))
        r_image[row_index, col_index] = cur_R #  + noise
        m_image[row_index, col_index] = region_ms[ttype]
        D_image[row_index, col_index] = region_Ds[ttype]
        

with open(PREPROCESSED_DATA + "SYNTHETIC/r_img.pickle", 'wb') as f:
    pickle.dump(r_image, f)
with open(PREPROCESSED_DATA + "SYNTHETIC/m_actual.pickle", 'wb') as f: 
    pickle.dump(m_image, f)
with open(PREPROCESSED_DATA + "SYNTHETIC/D_actual.pickle", 'wb') as f:
    pickle.dump(D_image, f)

## Synthetic image testing

Run ind model on synthetic data

Load image

In [ ]:
import numpy as np
import math
import os
import pickle

from model.models import *
from testing.run_inference import *
from preprocessing.generate_USGS_data import generate_image
from utils.plotting import *
from utils.constants import *

with open(PREPROCESSED_DATA + "SYNTHETIC/m_actual.pickle", 'rb') as F:
    m_actual = pickle.load(F)
with open(PREPROCESSED_DATA + "SYNTHETIC/D_actual.pickle", 'rb') as F:
    D_actual = pickle.load(F)
with open(PREPROCESSED_DATA + "SYNTHETIC/r_img.pickle", 'rb') as F:
    R_image = pickle.load(F)

# row_min = 1
# row_max = 15
# col_min = 1
# col_max = 15

# m_actual = m_actual[row_min:row_max,col_min:col_max,:]
# D_actual = D_actual[row_min:row_max,col_min:col_max,:]
# R_image = R_image[row_min:row_max,col_min:col_max,:]
print("Num pixels " + str(R_image.shape[0] * R_image.shape[1]))

Independent model 


In [ ]:
import time
start = time.time() 

m_est, D_est = ind_model(iterations=5,
                         image=R_image,
                         C=10,
                         V=50)
end = time.time()
mins = (end - start)/60
hours = mins/60
print("Took " + str(int(mins)) + " minutes, or " 
        + str(round(hours,2)) + " hours.")

EXP_NAME = "TEST_SYNTHETIC2"
if not os.path.exists('../output/' + EXP_NAME):
    os.makedirs('../output/' + EXP_NAME)
record_output(m_actual, D_actual, m_est, D_est, "ind/", EXP_NAME)

Segmentation Model

In [ ]:
m_est, D_est = seg_model(seg_iterations=40000, 
                            iterations=10, 
                            image=R_image,
                            C=10,
                            V=50)
record_output(m_actual, D_actual, m_est, D_est, "seg/", "TESTSEG")